# Make sure you use GPU (Graphical Processing Unit) runtime for this exercise.

"Runtime" menu > Change Runtime Type > Select "T4 GPU" from the "Hardware accelerator"

In [1]:
!pip install segment-geospatial

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.7/515.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.1/568.1 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.5/86.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import libraries
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

os - The os library provides a way to interact with the operating system, such as reading or writing files, and managing directories.

leafmap - a powerful Python library for interactive mapping and geospatial analysis. It is used for visualizing and analyzing geospatial data

samgeo - segment-geospatial

tms_to_geotiff - A function that converts Tile Map Service (TMS) data to GeoTIFF format.TMS delivers Map content as Web Map Tiles which can be viewed in all kinds of web, mobile, software, and GIS applications.
get_basemaps - A function to retrieve available basemaps

In [3]:
# Create and interactive map
m = leafmap.Map(center=(7.2936, 80.6411), zoom=18)

m.add_basemap("SATELLITE")
m

Map(center=[7.2936, 80.6411], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [4]:
# Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [
    80.6406,
    7.2931,
    80.6416,
    7.2941
]


In [5]:
# Download map tiles and mosaic them into a single GeoTIFF file
image = "satellite.tif"

In [6]:
# Specify the basemap as the source
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Downloaded image 01/16
Downloaded image 02/16
Downloaded image 03/16
Downloaded image 04/16
Downloaded image 05/16
Downloaded image 06/16
Downloaded image 07/16
Downloaded image 08/16
Downloaded image 09/16
Downloaded image 10/16
Downloaded image 11/16
Downloaded image 12/16
Downloaded image 13/16
Downloaded image 14/16
Downloaded image 15/16
Downloaded image 16/16
Saving GeoTIFF. Please wait...
Image saved to satellite.tif


In [7]:
# Display the downloaded image on the map
m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image, layer_name="Image")
m

Map(bottom=32191419.0, center=[7.2936, 80.6411], controls=(ZoomControl(options=['position', 'zoom_in_text', 'z…

In [8]:
# Initialize SAM class
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

Model checkpoint for vit_h not found.


Downloading...
From: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
To: /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth
100%|██████████| 2.56G/2.56G [00:13<00:00, 186MB/s]


vit_h - type of model used (Vision Transformer (ViT) model, type of deep learning model used for image analysis)

checkpoint - location of the file containing the model weights

sam_vit_h_4b8939.pth - file name of the checkpoint file

sam_kwargs=None - No additional arguements are passed. kwargs = key word arguments

In [10]:
# Segment the image
mask = "segment.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

100%|██████████| 4/4 [00:28<00:00,  7.05s/it]


image - input image

mask - file for saving the generated mask

batch=true - useful in handling large image or datasets

foreground=true - segmentation should focus on the foreground objects in the image. Those are the objects in the image that seem form its main part

erosion_kernel=(3, 3) - post processing operation which refine the edges of the segmented object

mask_multiplier=255 - Scale of the mask in 8-bit format, where pixel values range from 0 to 255.

In [11]:
# Polygonize the raster data. Save the segmentation results to a GeoPackage file
vector = "segment.gpkg"
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)


mask - file name of the input tiff

vector - file name of the output vector file

simplify_tolerance=None - Controls the simplication of the vector shapes. None means no simplification is applied. Simplification can reduce the complexity of the shapes by reducing the number of vertices.

In [12]:
# Saving segmentation results as any vector data format supported by GeoPandas
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

In [13]:
# Visualize results
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(vector, layer_name="Vector", style=style)
m

Map(bottom=32191419.0, center=[7.2936, 80.6411], controls=(ZoomControl(options=['position', 'zoom_in_text', 'z…

color - outline colour (blue)

Weight - outline thickness

fillcolor - fill colour (purple)

fill0pacity - transparency of the fill color. 0 is fully transparent and 1 is fully non-transparent

m - displays the map object

In [ ]:
# Download the Geopackage file.

# Import the necessary modules
from google.colab import files

# Download the Geopackage file
files.download(vector)

In [ ]:
# Download as a Shapefile